In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import joblib

In [17]:
df = pd.read_csv(r"D:\project\fertilizerrecommendation\data\f2.csv")

In [18]:
label_enc = LabelEncoder()
df['Fertilizer'] = label_enc.fit_transform(df['Fertilizer']) 

In [19]:
# Save the fitted label encoder for later use
joblib.dump(label_enc, r"D:\\project\\fertilizerrecommendation\\models\\label_encoder.pkl")

['D:\\\\project\\\\fertilizerrecommendation\\\\models\\\\label_encoder.pkl']

In [20]:
categorical_cols = [col for col in df.select_dtypes(include=['object']).columns if col != "Fertilizer"]
df = pd.get_dummies(df, columns=categorical_cols)


In [21]:
X = df.drop(columns=['Fertilizer'])
y = tf.keras.utils.to_categorical(df['Fertilizer'])

In [22]:
# Fit StandardScaler on numerical features before splitting
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [23]:
# Save the fitted scaler for later use
joblib.dump(scaler, r"D:\\project\\fertilizerrecommendation\\models\\scaler.pkl")

['D:\\\\project\\\\fertilizerrecommendation\\\\models\\\\scaler.pkl']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(y.shape[1], activation='softmax')
])

C:\Users\Naveen Reddy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.1029 - loss: 2.7919 - val_accuracy: 0.2613 - val_loss: 2.3921
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1425 - loss: 2.5357 - val_accuracy: 0.3153 - val_loss: 2.2402
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2092 - loss: 2.3192 - val_accuracy: 0.2342 - val_loss: 2.1065
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2782 - loss: 2.0883 - val_accuracy: 0.3153 - val_loss: 1.9823
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2994 - loss: 1.9841 - val_accuracy: 0.3333 - val_loss: 1.8786
Epoch 6/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3535 - loss: 1.9005 - val_accuracy: 0.3604 - val_loss: 1.7599
Epoch 7/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4194 - loss: 1.7830 - val_accuracy: 0.4324 - val_loss: 1.6646
Epoch 8/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4411 - loss: 1.6457 - val_accuracy: 0.4595 - val_loss

In [29]:
model.save(r"D:\project\fertilizerrecommendation\models\fertilizer.h5")
print("Model training complete and saved.")

Model training complete and saved.


In [30]:
import numpy as np
import pandas as pd

# Load the dataset
csv_file_path = r"D:\project\fertilizerrecommendation\data\f2.csv"
df = pd.read_csv(csv_file_path)

# Extract features and target column
features = ['Soil_Type', 'Crop_Type', 'Nitrogen', 'Potassium', 'Phosphorous']
target_column = 'Fertilizer'

# Convert categorical columns into numerical form (if necessary)
df_encoded = pd.get_dummies(df[features])

# Get unique fertilizer labels in order (for mapping index to fertilizer name)
unique_fertilizers = df[target_column].unique()

# Convert to NumPy array with dtype float32 (TensorFlow compatibility)
sample_input = df_encoded.iloc[:5].values.astype(np.float32)  # Select first 5 rows

# Ensure the model exists
if 'model' in globals() or 'model' in locals():
    try:
        # Make predictions
        pred = model.predict(sample_input)

        # Convert predictions from probability distributions to class labels
        predicted_indices = np.argmax(pred, axis=1)  # Get index of max probability

        # Map indices back to fertilizer names
        predicted_fertilizers = [unique_fertilizers[i] for i in predicted_indices]

        # Print the final fertilizer predictions
        print("Predicted Fertilizer Labels:", predicted_fertilizers)
    
    except Exception as e:
        print("Error while making predictions:", str(e))
else:
    print("Model not found. Please ensure it is properly defined and trained.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Predicted Fertilizer Labels: ['17-17-17', '17-17-17', '10-10-2010', '10-10-2010', '10-10-2010']
